In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import pickle
from emot.emo_unicode import UNICODE_EMOJI
import spacy
sp = spacy.load('en_core_web_sm')
import en_core_web_sm
nlp = en_core_web_sm.load()
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

bitcoin = pd.read_csv("bitcoin_3_month.csv")
btc = pd.read_csv("btc_3_month.csv")
bitcoin.head()


<ipython-input-1-0c3c0bea8c68>:15: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  bitcoin = pd.read_csv("bitcoin_3_month.csv")


id      conversation_id               created_at  \
0  1454233293109534720  1454233293109534720  2021-10-29 19:46:50 EDT   
1  1454232278092107780  1454229634648793094  2021-10-29 19:42:48 EDT   
2  1454231129804972042  1454231129804972042  2021-10-29 19:38:14 EDT   
3  1454230807233581061  1454229634648793094  2021-10-29 19:36:57 EDT   
4  1454229932255719425  1454220157220364288  2021-10-29 19:33:28 EDT   

         date      time  timezone             user_id         username  \
0  2021-10-29  19:46:50      -500  963767159536209921       theblock__   
1  2021-10-29  19:42:48      -500  823766058909761536  lynaldencontact   
2  2021-10-29  19:38:14      -500  729548255470575616    eric_lombrozo   
3  2021-10-29  19:36:57      -500  823766058909761536  lynaldencontact   
4  2021-10-29  19:33:28      -500            14110443           muneeb   

         name place  ... geo source user_rt_id user_rt retweet_id  \
0   The Block   NaN  ... NaN    NaN        NaN     NaN        NaN   
1   Lyn Alden   NaN  ... NaN    NaN        NaN     NaN        NaN   
2           👑   NaN  ... NaN    NaN        NaN     NaN        NaN   
3   Lyn Alden   NaN  ... NaN    NaN        NaN     NaN        NaN   
4  muneeb.btc   NaN  ... NaN    NaN        NaN     NaN        NaN   

                                            reply_to  retweet_date  translate  \
0                                                 []           NaN        NaN   
1  [{'screen_name': 'Croesus_BTC', 'name': 'Croes...           NaN        NaN   
2                                                 []           NaN        NaN   
3  [{'screen_name': 'skwp', 'name': 'Yan Pritzker...           NaN        NaN   
4  [{'screen_name': 'jack', 'name': 'jack⚡️', 'id...           NaN        NaN   

  trans_src trans_dest  
0       NaN        NaN  
1       NaN        NaN  
2       NaN        NaN  
3       NaN        NaN  
4       NaN        NaN  

[5 rows x 36 columns]

In [2]:
# select some column
bitcoin = bitcoin[['id','date','time','tweet','language','replies_count','retweets_count', 'likes_count']]
btc = btc[['id','created_at','tweet','language','replies_count','retweets_count', 'likes_count']]
print(bitcoin.shape)
print(btc.shape)

# # drop ADS???
bitcoin_drop = bitcoin.drop_duplicates(keep=False)
btc_drop = btc.drop_duplicates(keep=False)
print(bitcoin_drop.shape)
print(btc_drop.shape)

# bitcoin_btc = bitcoin_drop.merge(btc_drop, on='id', how='outer', indicator='btc_status')
bitcoin_btc  = pd.concat([bitcoin_drop, btc_drop],
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=False,
)
bitcoin_btc = bitcoin_btc.drop_duplicates(subset='id')
print(bitcoin_btc.shape)
bitcoin_btc

(46566, 8)
(45581, 7)
(41328, 8)
(596, 7)
(41820, 9)


id        date      time  \
172    1454132785908797441  2021-10-29  13:07:27   
173    1454132298983649295  2021-10-29  13:05:31   
174    1454132043739185162  2021-10-29  13:04:30   
175    1454131968132648971  2021-10-29  13:04:12   
176    1454131699290357764  2021-10-29  13:03:08   
...                    ...         ...       ...   
45238  1486874424364183556         NaN       NaN   
45239  1486871174164324352         NaN       NaN   
45240  1486868220413984771         NaN       NaN   
45241  1486865585774678016         NaN       NaN   
45242  1486860682951860225         NaN       NaN   

                                                   tweet language  \
172    7/ Snow Crash (1992) predicted digital currenc...       en   
173    For one day only, I am accepting bitcoin from ...       en   
174                    @gator_bitcoin Do you own floki ?       en   
175    @PizzaAtlanta @JoePompliano @BitcoinPizza_io @...       en   
176    @jacque_davis I would be blind with panic if I...       en   
...                                                  ...      ...   
45238  @WellbourneM Once the $btc price stabilizes, A...       en   
45239  @JuanSGuarnizo TO THE MOON $BTC $ETH $USDT $EL...       en   
45240  @BosRec I fought this kind of FUD from BTC max...       en   
45241  BREAKING: Candidate for Governor of #Texas lay...       en   
45242  @tom49coffee @PrfDude Maybe.  Consider thought...       en   

       replies_count  retweets_count  likes_count               created_at  
172                2               8           55                      NaN  
173                0               0            0                      NaN  
174                0               0            1                      NaN  
175                0               0            4                      NaN  
176                1               0            0                      NaN  
...              ...             ...          ...                      ...  
45238              2               0            5  2022-01-27 20:31:02 EST  
45239              3               3          395  2022-01-27 20:18:07 EST  
45240              0               0           21  2022-01-27 20:06:23 EST  
45241              8               0            9  2022-01-27 19:55:54 EST  
45242              0               0            2  2022-01-27 19:36:26 EST  

[41820 rows x 9 columns]

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chayisara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# language EN only
df = bitcoin_btc[bitcoin_btc['language'] == 'en'].reset_index()
print(df["tweet"].shape)
df['tweet'] = df['tweet'].astype(str)
tweet = df[['id','tweet']]

(41773,)


In [5]:
# for i in tweet:
#     spacy_tweet = nlp(i)
# #     print(type(spacy_tweet))
#     for one_tok in spacy_tweet.ents:
#         print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)

In [6]:
# def get_money(one_string):
    
#     tagged_str = nlp(one_string) 
#     all_money = [one_tok.text for one_tok in tagged_str.ents 
#                   if one_tok.label_ == "MONEY"]
#     return(all_money)

# all_money = [get_money(one_string) for one_string in tweet]
# all_money

In [7]:
# apply all lower case
# remove URL "https://t.co/"
df_lower = pd.DataFrame(df["tweet"].str.lower())
# print(df_lower["tweet"])
url_pattern = r"https://t.co/[A-Za-z0-9]+"
new_url = [re.sub(url_pattern, " ", tweet) for tweet in df_lower.tweet]
new_url[4]

# # emoji

"@jacque_davis i would be blind with panic if i realized it were worth that. oh and those stories about people who lost the password they need to unlock like $200 million in bitcoin? my god. i'd run up a massive therapy bill."

In [8]:
# remove @mention
mention_pattern = r"@[A-Za-z0-9!#%&*;_\$\.]+"
new_mention = [re.sub(mention_pattern, " ", tweet) for tweet in new_url]
print(new_mention[4])

  i would be blind with panic if i realized it were worth that. oh and those stories about people who lost the password they need to unlock like $200 million in bitcoin? my god. i'd run up a massive therapy bill.


In [9]:
# remove # ,turning hashtags to the typical words.
new_symbol = [re.sub(r"\W+", " ", tweet) for tweet in new_mention]
print(new_symbol[4])

 i would be blind with panic if i realized it were worth that oh and those stories about people who lost the password they need to unlock like 200 million in bitcoin my god i d run up a massive therapy bill 


In [10]:
# remove stopword
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

new_stopwords = remove_stop_words(new_symbol)
new_stopwords[4]

'would blind panic realized worth oh stories people lost password need unlock like 200 million bitcoin god run massive therapy bill'

In [11]:
# reduce character sequences >3 to 3
sequence_pattern = re.compile(r"(\w)\1+")
def repl(matchObj):
   char = matchObj.group(1)
   return "%s%s" % (char, char)

new_sequence = [re.sub(sequence_pattern, repl, tweet) for tweet in new_stopwords]
new_sequence[4]

'would blind panic realized worth oh stories people lost password need unlock like 200 million bitcoin god run massive therapy bill'

In [12]:
# remove 2-character words
new_twochar = [re.sub(r"\b[a-z]{1,2}\b", " ", tweet) for tweet in new_sequence]
new_twochar[4]

'would blind panic realized worth   stories people lost password need unlock like 200 million bitcoin god run massive therapy bill'

In [13]:
# replace 2 or more spaces with a single space.
new_space = [re.sub(r"\s+", " ", tweet) for tweet in new_twochar]
new_space[4]

'would blind panic realized worth stories people lost password need unlock like 200 million bitcoin god run massive therapy bill'

In [14]:
# remove spaces front and back
new_space_end = [re.sub(r"^\s+|\s$", "", tweet) for tweet in new_space]
new_space_end[4]

'would blind panic realized worth stories people lost password need unlock like 200 million bitcoin god run massive therapy bill'

In [15]:
sent_obj = SentimentIntensityAnalyzer()
print(type(sent_obj))

sent_data = new_space_end
sentiment_example = []
for i in sent_data:
    sentiment_example.append(sent_obj.polarity_scores(i))
sentiment_example

<class 'vaderSentiment.vaderSentiment.SentimentIntensityAnalyzer'>


[{'neg': 0.251, 'neu': 0.512, 'pos': 0.237, 'compound': -0.0772},
 {'neg': 0.123, 'neu': 0.71, 'pos': 0.168, 'compound': 0.1779},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.6369},
 {'neg': 0.288, 'neu': 0.486, 'pos': 0.226, 'compound': -0.4215},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.795, 'pos': 0.205, 'compound': 0.4767},
 {'neg': 0.094, 'neu': 0.906, 'pos': 0.0, 'compound': -0.1695},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.698, 'pos': 0.302, 'compound': 0.8555},
 {'neg': 0.037, 'neu': 0.488, 'pos': 0.476, 'compound': 0.9371},
 {'neg': 0.0, 'neu': 0.748, 'pos': 0.252, 'compound': 0.4019},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.237, 'neu': 0.536, 'pos': 0.228, 'compound': -0.0516},
 {'neg': 0.0, 'neu': 0.812, 'pos': 0.188, 'compound': 0.5267},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu

In [16]:
# sent_obj.lexicon
neg = [sent_obj.lexicon[k] for k in sent_obj.lexicon if sent_obj.lexicon[k] < 0]
np.mean(neg)
pos = [sent_obj.lexicon[k] for k in sent_obj.lexicon if sent_obj.lexicon[k] > 0]
np.mean(pos)

# sent_obj.lexicon['']

-1.552026864955625

1.5321845969433623

In [17]:
pest_words = {
    'up': 2.0,
    'down': -2.0,
    'green' : 2.0,
    'red' : -2.0,
    'bull' : 2.0,
    'bear' : -2.0,
    'buy' : 2.0,
    'bought' : 2.0,
    'sell' : -2.0,
    'sold' : -2.0
}


## initiate new sentiment object
## so that we don't alter old one
## use.update to add new words
new_si = SentimentIntensityAnalyzer()
new_si.lexicon.update(pest_words)

## try re-scoring the third example
## see negative
sentiment_update = []
for i in sent_data:
    sentiment_update.append(sent_obj.polarity_scores(i))
sentiment_update

[{'neg': 0.251, 'neu': 0.512, 'pos': 0.237, 'compound': -0.0772},
 {'neg': 0.123, 'neu': 0.71, 'pos': 0.168, 'compound': 0.1779},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.6369},
 {'neg': 0.288, 'neu': 0.486, 'pos': 0.226, 'compound': -0.4215},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.795, 'pos': 0.205, 'compound': 0.4767},
 {'neg': 0.094, 'neu': 0.906, 'pos': 0.0, 'compound': -0.1695},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.698, 'pos': 0.302, 'compound': 0.8555},
 {'neg': 0.037, 'neu': 0.488, 'pos': 0.476, 'compound': 0.9371},
 {'neg': 0.0, 'neu': 0.748, 'pos': 0.252, 'compound': 0.4019},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.237, 'neu': 0.536, 'pos': 0.228, 'compound': -0.0516},
 {'neg': 0.0, 'neu': 0.812, 'pos': 0.188, 'compound': 0.5267},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu

In [18]:
all_compound = [one_tok['compound'] for one_tok in sentiment_update]
temp = pd.DataFrame(all_compound)
df['compound'] = temp
df

index                   id        date      time  \
0        172  1454132785908797441  2021-10-29  13:07:27   
1        173  1454132298983649295  2021-10-29  13:05:31   
2        174  1454132043739185162  2021-10-29  13:04:30   
3        175  1454131968132648971  2021-10-29  13:04:12   
4        176  1454131699290357764  2021-10-29  13:03:08   
...      ...                  ...         ...       ...   
41768  45238  1486874424364183556         NaN       NaN   
41769  45239  1486871174164324352         NaN       NaN   
41770  45240  1486868220413984771         NaN       NaN   
41771  45241  1486865585774678016         NaN       NaN   
41772  45242  1486860682951860225         NaN       NaN   

                                                   tweet language  \
0      7/ Snow Crash (1992) predicted digital currenc...       en   
1      For one day only, I am accepting bitcoin from ...       en   
2                      @gator_bitcoin Do you own floki ?       en   
3      @PizzaAtlanta @JoePompliano @BitcoinPizza_io @...       en   
4      @jacque_davis I would be blind with panic if I...       en   
...                                                  ...      ...   
41768  @WellbourneM Once the $btc price stabilizes, A...       en   
41769  @JuanSGuarnizo TO THE MOON $BTC $ETH $USDT $EL...       en   
41770  @BosRec I fought this kind of FUD from BTC max...       en   
41771  BREAKING: Candidate for Governor of #Texas lay...       en   
41772  @tom49coffee @PrfDude Maybe.  Consider thought...       en   

       replies_count  retweets_count  likes_count               created_at  \
0                  2               8           55                      NaN   
1                  0               0            0                      NaN   
2                  0               0            1                      NaN   
3                  0               0            4                      NaN   
4                  1               0            0                      NaN   
...              ...             ...          ...                      ...   
41768              2               0            5  2022-01-27 20:31:02 EST   
41769              3               3          395  2022-01-27 20:18:07 EST   
41770              0               0           21  2022-01-27 20:06:23 EST   
41771              8               0            9  2022-01-27 19:55:54 EST   
41772              0               0            2  2022-01-27 19:36:26 EST   

       compound  
0       -0.0772  
1        0.1779  
2        0.0000  
3        0.6369  
4       -0.4215  
...         ...  
41768    0.0000  
41769    0.0000  
41770   -0.0516  
41771    0.1280  
41772    0.0000  

[41773 rows x 11 columns]

In [19]:
# # weight retweet
# df['famous_score'] = df['replies_count']*1/3 + df['retweets_count']*1/3 + df['likes_count']*1/3
# df['famous_score'].describe()
# df['famous_score'][df['famous_score'] < 50].count()
# # 50
# for i in 
# # 50 - 200
# # 200 - 500
# # 500 - 999
# # 1000 - 3000
# # 3000 - 9999
# # 9999


SyntaxError: invalid syntax (<ipython-input-19-98dad792adba>, line 6)

In [ ]:
# # create date-time column
# df_bydate = df.groupby(['date']).agg({"tweet" : "sum"}).reset_index()
# df_bydate[:20]